In [31]:
import numpy as np
from scipy.linalg import eigh
from functools import reduce
import matplotlib.pyplot as plt

In [32]:
# 系统参数
N = 10  # qubit数
J = 1.0  # 自旋耦合强度
h = 1.0  # 横场强度


In [33]:
# 定义泡利矩阵
I = np.eye(2)
X = np.array([[0, 1], [1, 0]])
Z = np.array([[1, 0], [0, -1]])

In [34]:
# 构造N体泡利矩阵的张量积
def kron_N(pauli_list):
    return reduce(np.kron, pauli_list)

In [35]:
# 构造Hamiltonian矩阵
def build_hamiltonian(N, J, h):
    H = np.zeros((2**N, 2**N))
    # 最近邻Z_i Z_{i+1}项
    for i in range(N - 1):
        ops = [I] * N
        ops[i] = Z
        ops[i + 1] = Z
        H -= J * kron_N(ops)
    # 横场X_i项
    for i in range(N):
        ops = [I] * N
        ops[i] = X
        H -= h * kron_N(ops)
    return H

In [36]:
# 构造总磁化算符
def build_total_op(N, op):
    total = np.zeros((2**N, 2**N))
    for i in range(N):
        ops = [I] * N
        ops[i] = op
        total += kron_N(ops)
    return total

In [37]:
# 构建Hamiltonian
H = build_hamiltonian(N, J, h)

In [38]:
# 对角化
eigvals, eigvecs = eigh(H)
E0 = eigvals[0]
ground_state = eigvecs[:, 0]

In [39]:
# 构建总磁化算符
Z_total = build_total_op(N, Z)
X_total = build_total_op(N, X)

In [41]:
# 计算期望值
mz = np.vdot(ground_state, Z_total @ ground_state).real / N
mx = np.vdot(ground_state, X_total @ ground_state).real / N

# 输出结果
print(f"Exact Diagonalization Results for N={N}")
print(f"Ground state energy E0: {E0:.6f}")
print(f"<Z> per site: {mz:.6f}")
print(f"<X> per site: {mx:.6f}")

Exact Diagonalization Results for N=10
Ground state energy E0: -12.381490
<Z> per site: -0.000000
<X> per site: 0.732255


In [ ]:
def magnetization_x(ground_state, N, X, kron_N, I):
    M = 0.0
    for i in range(N):
        ops = [I] * N
        ops[i] = X
        op = kron_N(ops)
        M += np.vdot(ground_state, op @ ground_state).real
    return M / N

In [ ]:
h_list = np.logspace(-2, 2, 30)
N_list = range(2, 11)
mx_vs_h = np.zeros((len(N_list), len(h_list)))
for iN, N in enumerate(N_list):
    for ih, h in enumerate(h_list):
        H = build_hamiltonian(N, J=1.0, h=h)
        eigvals, eigvecs = eigh(H)
        ground_state = eigvecs[:, 0]
        mx = magnetization_x(ground_state, N, X, kron_N, I)
        mx_vs_h[iN, ih] = mx

plt.figure(figsize=(8,6))
for iN, N in enumerate(N_list):
    plt.plot(h_list, mx_vs_h[iN], label=f"N={N}")
plt.xscale('log')
plt.xlabel('Transverse Field h (log scale)')
plt.ylabel(r'$\langle X \rangle$')
plt.title(r'Magnetization $\langle X \rangle$ vs $h$ for $N=2$ to $10$ (J=1)')
plt.legend()
plt.grid(True, which='both', ls='--', alpha=0.5)
plt.show()